# B-tree Implementation for Key-value Store
#### An example project illustring B-tree implementation in a key-value store with numeric/non-numeric unique keys including functional and performance tests

### Node Implementation

In [136]:
import bisect

class Node:
    
    def __init__(self):
        self.keys = []
        self.values = []
        self.parent = None
        self.children = None
        
    def _get_value(self, key):
        for i, k in enumerate(self.keys):
            if k == key:
                return self.values[i]
        return None

    def _get_insert_index(self, key):
        insert_index = bisect.bisect(self.keys, key)
        return insert_index
    
    def _insert_entry(self, key, value):
        insert_index = self._get_insert_index(key)
        self.keys.insert(insert_index, key)
        self.values.insert(insert_index, value)

    def _update_entry(self, key, value):
        for i, k in enumerate(self.keys):
            if k == key:
                self.values[i] = value        
                
    def add_update(self, key, value):
        if not self._get_value(key):
            self._insert_entry(key, value)
        else:
            self._update_entry(key, value)

### B-tree Implementation

In [25]:
class BTree:
    
    def __init__(self):
        self.root = Node()
        self.height = 0
        self.size = 0
        
        
    def _add(self, current_node, key, value):
        

['a', 'b', 'c', 'd']

In [137]:
x = Node()


In [139]:
x.add_update('k', 4)

In [140]:
x.keys

['k']

In [141]:
x.values

[4]

In [142]:
x.add_update('a', 5)

In [143]:
x.keys

['a', 'k']

In [144]:
x.values

[5, 4]

In [145]:
x._get_value('k')

4

In [146]:
x._get_value('b')

In [148]:
x.add_update('b', 19)

In [149]:
x.keys

['a', 'b', 'k']

In [150]:
x.values

[5, 19, 4]

In [151]:
x.add_update('b', 15)

In [152]:
x.values

[5, 15, 4]

In [153]:
x.keys

['a', 'b', 'k']

In [154]:
x._get_value('z')